# Instalo e importo librerías

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=9dabf6a8c2808c2614f22182287890c11a2d0eeacfbef10d64a9f6f871eef287
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected packag

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Google Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Creo sesión de spark y leo csv

In [ ]:
downloaded = drive.CreateFile({'id':"10xgOf2rORcGlcKPME3cHGHssXGzNemer"})
downloaded.GetContentFile('GooglePlayStore_User_Reviews.csv')

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('GooglePlayStore_User_Reviews.csv', header=True, inferSchema=True)
user_reviews = df.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Ejercicio 2

Teniendo en cuenta las reviews que reciben las aplicaciones, devolver una (al azar) de la aplicación que haya recibido la mayor cantidad de reviews positivas. (⭐)

Elimino las reviews nan o None ya que las considero un error del archivo (no tienen sentiment ya que sin review no se puede saber esto).

Uso distinct para eliminar reviews que se encuentran repetidas (aparecen con todos los datos iguales y provoca que algunas apps terminen sumando más reviews positivas que las que realmente tienen).

Mapeo: me quedo con app como clave y como valor (review, ToF): True si la review es positiva, False si no.

Luego, sumo la cantidad de reviews positivas y me voy quedando con una review por app (como el orden de ejecución no está asegurado, es una review al azar; si queremos que el resultado de distinto, podríamos hacer random.choice([x[0], y[0]]) en lugar de quedarnos con x[0] en el reduceByKey).

Finalmente, obtengo la app con más positivas (con un reduce) y devuelvo una review de esta (podría mostrarlo como clave-valor pero decidí mostrar sólo la review).

In [ ]:
user_reviews.filter(lambda x: (x.Translated_Review != 'nan') & (x.Translated_Review is not None)).distinct().map(lambda x: (x.App, (x.Translated_Review, x.Sentiment == 'Positive')))\
.reduceByKey(lambda x,y: (x[0], x[1] + y[1]))\
.reduce(lambda x,y: x if x[1][1] > y[1][1] else y)[1][0] #[1][0] par ver solo review, si queremos ver app y cantidad de positivas, lo sacamos

"This best note system I've ever used! Easy simple productive tool. I've used keep records health history, wrote novel, kept addresses, shopping notes, receipts, grocery lists, anything copy paste included here, even lock notes password, keep passwords recorded behind simple password, keep organized colored notes, share. I love it! If phone breaks, worries... It's backed up! There drawbacks here!"